In [1]:
import numpy as np
import pandas as pd
import biocircuits 
import scipy.integrate

import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

# <center> Homework 2.2: XOR Gates </center>

# a)
\begin{align}
f(x) = \cfrac{x^{n_x} + y^{n_y}}{(1+x^{n_x})(1+y^{n_y})}
\end{align}

<!-- Q: Based on this expression what would have to be true of the promoter architecture in terms of how the effectors may bind? Do you think this scenario would be well-modeled by the regulatory function you wrote?
 -->
 
X and Y together inhibit the production of Z, but alone allow for its production. Physiologically, I don't really understand why this might happen, but here are some ideas: 

- Sequestration: when X and Y are both present, X can sequester Y either physically (via molecular cages or some reaction that changes its phase or solubility, or chemically by deactivating important binding regions. The problem here is that even if you have any leftover X or Y, there will always be some basal level of expression.

- Not enough space! There is simply not enough space for both to bind, in which case my denominator would be wrong (the $x^{n_x}y^{n_y}$ should be zero). Another space consideration I thought of was that if X and Y had two adjacent binding regions, there could be side group (chemical groups not related to the X-Z or Y-Z binding region) interactions between X-Y, which make X induce a conformational change that unhinges Y, or vice versa. But either way, with this interpretaion, when constructing our "states and weights", we would effectively assign a weight of 0 to $x^{n_x}y^{n_y}$ both being bound, and the denominator would be wrong. So given the above, we must allow both to bind. 

- Both can bind: The most mathematically sound interpretation of the equation written above is that X and Y both recruit the polymerase on their own, but when both are bound, the polymerase is physically blocked in some way. I remember learning from an Ed post/response that heterochromatin/euchromatin ([YouTube animation here!](https://www.youtube.com/watch?v=Tn5qgEqWgW8&ab_channel=AidenLab)) can bundle/unwind the DNA. I'm not sure when this happens, or what the timescales of this are, but theoretically, I can imagine that X and Y together initiate some kind of folding process in the DNA, and prevents it from actually being read. 

<div class="alert alert-block alert-info">
    These are good ideas!
</div>

# b)
<!-- b) Sketch a genetic circuit that can function as an XOR gate. That is, X and Y are inputs and the output is the concentration of a single gene product Z. You will likely need to have more than just X, Y, and Z in your circuit. Clearly explain the thinking behind the circuit you chose. Are there feedforward loops in your circuit? How about feedback loops?
 -->
So far the approach that seems to be able to be confirmed with truth tables consists strictly of feedforward loops (the feedback attempt is not shown.)

I approached this problem in two steps: first, using some computation to combinatorically see how many extra species I needed to add / ruling out why simple combinations of activators/reppressors wouldn't work. I played around with a couple gates until I saw it was possible by adding X and ¬X and Y and ¬Y, then tried to turn this into a circuit. 

In [2]:
X = np.array([0, 0, 1, 1])
Y = np.array([0, 1, 0, 1])

NOT_X = np.array([1, 1, 0, 0])
NOT_Y = np.array([1, 0, 1, 0])

AND = np.logical_and
OR = np.logical_or

# 👀 🔍 searching for 1️⃣0️⃣0️⃣1️⃣...

<div class="alert alert-block alert-info">
    this cute
</div>

We can see why we need a third element (we cannot just take pair-wise combinations) with one gate. 

In [3]:
from itertools import combinations, permutations

for first, second in list(combinations([X, Y, NOT_X, NOT_Y], 2)):
    print(first, second, end='\t\t\t')
    print("AND:", AND(first, second)*1, end='    ')
    print("OR:", OR(first, second)*1)    

[0 0 1 1] [0 1 0 1]			AND: [0 0 0 1]    OR: [0 1 1 1]
[0 0 1 1] [1 1 0 0]			AND: [0 0 0 0]    OR: [1 1 1 1]
[0 0 1 1] [1 0 1 0]			AND: [0 0 1 0]    OR: [1 0 1 1]
[0 1 0 1] [1 1 0 0]			AND: [0 1 0 0]    OR: [1 1 0 1]
[0 1 0 1] [1 0 1 0]			AND: [0 0 0 0]    OR: [1 1 1 1]
[1 1 0 0] [1 0 1 0]			AND: [1 0 0 0]    OR: [1 1 1 0]


Let's try to see if summing 3 elements with one gate will work.

In [4]:
for first, second, third in list(combinations([X, Y, NOT_X, NOT_Y], 3)):
    print(first, second, third, end='\t\t')
    print("AND:", AND(AND(first, second), third)*1, end='  ')
    print("OR:", OR(OR(first, second), third)*1*1, end='  ')    
    print("AND-OR:", AND(OR(first, second), third)*1, end='  ')
    print("OR-AND:", OR(AND(first, second), third)*1) 

[0 0 1 1] [0 1 0 1] [1 1 0 0]		AND: [0 0 0 0]  OR: [1 1 1 1]  AND-OR: [0 1 0 0]  OR-AND: [1 1 0 1]
[0 0 1 1] [0 1 0 1] [1 0 1 0]		AND: [0 0 0 0]  OR: [1 1 1 1]  AND-OR: [0 0 1 0]  OR-AND: [1 0 1 1]
[0 0 1 1] [1 1 0 0] [1 0 1 0]		AND: [0 0 0 0]  OR: [1 1 1 1]  AND-OR: [1 0 1 0]  OR-AND: [1 0 1 0]
[0 1 0 1] [1 1 0 0] [1 0 1 0]		AND: [0 0 0 0]  OR: [1 1 1 1]  AND-OR: [1 0 0 0]  OR-AND: [1 1 1 0]


interesting result, but no cigar!

Let's try mixing and matching. 

In [5]:
for A, B in list(combinations([X, Y, NOT_X, NOT_Y], 2)):
    print(A, B, end='\t\t\n')
    print( '\t', (AND(AND(X, Y), AND(A, B))) * 1, end=' '), 
    print( (AND(AND(X, Y), OR(A, B))) * 1, end=' '),     
    print( (AND(OR(X, Y), AND(A, B))) * 1, end=' '), 
    end= ' '
    if ((AND(OR(X, Y), OR(A, B))) * 1==np.array([0,1,1,0])).all(): end="*"
    print( (AND(OR(X, Y), OR(A, B))) * 1, end=end),     
    print( '\n\t', (OR(AND(X, Y), AND(A, B))) * 1, end=' '), 
    print( (OR(AND(X, Y), OR(A, B))) * 1, end=' '),     
    print( (OR(OR(X, Y), AND(A, B))) * 1, end=' '),     
    print( (OR(OR(X, Y), OR(A, B))) * 1),    

[0 0 1 1] [0 1 0 1]		
	 [0 0 0 1] [0 0 0 1] [0 0 0 1] [0 1 1 1] 
	 [0 0 0 1] [0 1 1 1] [0 1 1 1] [0 1 1 1]
[0 0 1 1] [1 1 0 0]		
	 [0 0 0 0] [0 0 0 1] [0 0 0 0] [0 1 1 1] 
	 [0 0 0 1] [1 1 1 1] [0 1 1 1] [1 1 1 1]
[0 0 1 1] [1 0 1 0]		
	 [0 0 0 0] [0 0 0 1] [0 0 1 0] [0 0 1 1] 
	 [0 0 1 1] [1 0 1 1] [0 1 1 1] [1 1 1 1]
[0 1 0 1] [1 1 0 0]		
	 [0 0 0 0] [0 0 0 1] [0 1 0 0] [0 1 0 1] 
	 [0 1 0 1] [1 1 0 1] [0 1 1 1] [1 1 1 1]
[0 1 0 1] [1 0 1 0]		
	 [0 0 0 0] [0 0 0 1] [0 0 0 0] [0 1 1 1] 
	 [0 0 0 1] [1 1 1 1] [0 1 1 1] [1 1 1 1]
[1 1 0 0] [1 0 1 0]		
	 [0 0 0 0] [0 0 0 0] [0 0 0 0] [0 1 1 0]*
	 [1 0 0 1] [1 1 1 1] [1 1 1 1] [1 1 1 1]


**We found one! it is one above the bottom-right corner, marked \* for convenience.** 

By looking at the arrays and the positioning of the print statements, this is the system using: X, Y, ¬X, ¬Y, with the following logic: 

<center>Z = (X ∨ Y) ∧ (¬X ∨ ¬Y) = ((X ∧ ¬X) ∨ (X ∧ ¬Y)) ∨ ((Y ∧ ¬X) ∨ (Y ∧ ¬Y)). </center>
<center> = (X ∧ ¬Y) ∨ (Y ∧ ¬X)

In words, this means Z is an OR gate, with a species before with an AND gate with X as an activator, Y as a repressor, and another species with an AND gate with Y as an activator, and X as a repressor. It looks like the following: 
    
<img src="XOR_circuit.jpg">


# c) 
 The system of equations we write down are a probabilistic interpretation of our states and weights derived in class. Recall in the AND gate, requiring both X and Y to be bound means taking the product of those weights, since they both must happen. Here, A and B are both AND gates but require X to be bound and Y to not, and vice versa. We thus arrive at the following:
 
\begin{align}
\cfrac{dA}{dt} = \beta_A \cfrac{X^{n_{XA}}(1-Y^{n_{YA}})}{(1+X^{n_{XA}})(1+Y^{n_{YA}})} - \gamma_A A \\[0.5em]
\cfrac{dB}{dt} = \beta_B \cfrac{Y^{n_{YB}}(1-X^{n_{XB}})}{(1+X^{n_{XB}})(1+Y^{n_{YB}})} - \gamma_B B \\[0.5em]
\cfrac{dZ}{dt} = \beta_Z \cfrac{A^{n_{AZ}} + B^{n_{BZ}}}{(1+A^{n_{AZ}})(1+B^{n_{BZ}})} - \gamma_Z Z \\[0.5em]
\end{align}

Let's make a dashing DASHBOARD

In [6]:
def derivs(ABZ, t, t_step, gamma_array, beta_array, n_array, step):
    A, B, Z = ABZ
    if t < t_step: 
        X = step[0]
        Y = step[1]
    if t >= t_step: 
        X = step[2]
        Y = step[3]

    gammaA, gammaB, gammaZ = gamma_array
    betaA, betaB, betaZ = beta_array
    nXA, nXB, nYA, nYB, nAZ, nBZ = n_array

    deriv_A = betaA*(X**nXA*(1-Y**nYA))/((1+X**nXA)*(1+Y**nYA)) - gammaA*A
    deriv_B = betaB*(Y**nYB*(1-X**nXB))/((1+X**nXB)*(1+Y**nYB)) - gammaB*B
    deriv_Z = betaZ*(A**nAZ + B**nBZ)/((1+A**nAZ)*(1+B**nBZ)) - gammaZ*Z
    
    return np.array([deriv_A, deriv_B, deriv_Z])

In [7]:
gammaA_slider = pn.widgets.FloatSlider(name="γ_A", start=0.1, end=5.0, value=1.0, width=100)
gammaB_slider = pn.widgets.FloatSlider(name="γ_B", start=0.1, end=5.0, value=1.0, width=100)
gammaZ_slider = pn.widgets.FloatSlider(name="γ_Z", start=0.1, end=5.0, value=1.0, width=100)

betaA_slider = pn.widgets.FloatSlider(name="β_A", start=0.1, end=10.0, value=1.0, width=100)
betaB_slider = pn.widgets.FloatSlider(name="β_B", start=0.1, end=10.0, value=1.0, width=100)
betaZ_slider = pn.widgets.FloatSlider(name="β_Z", start=0.1, end=10.0, value=1.0, width=100)

nXA_slider = pn.widgets.FloatSlider(name="n_XA", start=0.1, end=15.0, value=5.0, width=100)
nXB_slider = pn.widgets.FloatSlider(name="n_XB", start=0.1, end=15.0, value=5.0, width=100)
nYA_slider = pn.widgets.FloatSlider(name="n_YA", start=0.1, end=15.0, value=5.0, width=100)
nYB_slider = pn.widgets.FloatSlider(name="n_YB", start=0.1, end=15.0, value=5.0, width=100)
nAZ_slider = pn.widgets.FloatSlider(name="n_AZ", start=0.1, end=15.0, value=5.0, width=100)
nBZ_slider = pn.widgets.FloatSlider(name="n_BZ", start=0.1, end=15.0, value=5.0, width=100)

t_step_slider = pn.widgets.FloatSlider(name="step time", start=0.0, end=5.0, value=1.8, width=300)
Xo_button = pn.widgets.RadioButtonGroup(name="Xo", options=["Xo LOW","Xo HIGH"], value="Xo LOW")
Yo_button = pn.widgets.RadioButtonGroup(name="Yo", options=["Yo LOW","Yo HIGH"], value="Yo LOW")

Xf_button = pn.widgets.RadioButtonGroup(name="Xf", options=["Xf LOW","Xf HIGH"], value="Xf LOW")
Yf_button = pn.widgets.RadioButtonGroup(name="Yf", options=["Yf LOW","Yf HIGH"], value="Yf HIGH")

normalize_button = pn.widgets.RadioButtonGroup(
    name="normalize", options=["NORMALIZE HALF", "NORMALIZE OFF"], value="NORMALIZE HALF")

In [8]:
@pn.depends(gammaA_slider.param.value, gammaB_slider.param.value, gammaZ_slider.param.value,
            betaA_slider.param.value, betaB_slider.param.value, betaZ_slider.param.value,
            nXA_slider.param.value, nXB_slider.param.value, nYA_slider.param.value,
            nYB_slider.param.value, nAZ_slider.param.value, nBZ_slider.param.value,
            Xo_button.param.value, Yo_button.param.value, Xf_button.param.value, Yf_button.param.value,
            t_step_slider.param.value, normalize_button.param.value
           )
def plotter(gammaA, gammaB, gammaZ, 
            betaA, betaB, betaZ, 
            nXA, nXB, nYA, nYB, nAZ, nBZ, 
            Xo_type, Yo_type, Xf_type, Yf_type,
            t_step, normalize
           ):
    Xo, Yo, Xf, Yf = 0, 0, 0, 0
    if Xo_type == "Xo HIGH": Xo = 1
    if Yo_type == "Yo HIGH": Yo = 1
    if Xf_type == "Xf HIGH": Xf = 1
    if Yf_type == "Yf HIGH": Yf = 1

    ABZo = np.array([0.0, 0.0, 0.0])
    t = np.linspace(0, 10, 500)

    gamma_array = np.array([gammaA, gammaB, gammaZ])
    beta_array = np.array([betaA, betaB, betaZ])
    n_array = np.array([nXA, nXB, nYA, nYB, nAZ, nBZ])
    step = [Xo, Yo, Xf, Yf]
    args = (t_step, gamma_array, beta_array, n_array, step)
    
    # .... integrating .... 
    ABZ = scipy.integrate.odeint(derivs, ABZo, t, args=args)
    A, B, Z = ABZ.T
    if normalize == "NORMALIZE HALF":
        if len(A[A==0]) != len(A): A /= (A.max()*2)
        if len(B[B==0]) != len(B): B /= (B.max()*2)
        if len(Z[Z==0]) != len(Z): Z /= (Z.max()*2)
    X, Y = np.empty(len(t)), np.empty(len(t))
    X[t < t_step] = Xo
    X[t >= t_step] = Xf
    Y[t < t_step] = Yo
    Y[t >= t_step] = Yf

    p = bokeh.plotting.figure(height=400, width=610, title="X[PL]OR", 
                              x_axis_label="time", y_axis_label="[ ]")
    p.line(t, X, line_width=3, color="red", legend_label="X",line_dash="dashdot")
    p.line(t, Y, line_width=3, color="orangered", legend_label="Y", line_dash="dotdash")

    p.line(t, A, line_width=3, color="#eba8b5", legend_label="A")
    p.line(t, B, line_width=3, color="#9fc0c1", legend_label="B")
    p.line(t, Z, line_width=3, color="#65042d", legend_label="Z")
    
    p.legend.click_policy="hide"
    p.legend.location="top_left"


    return style(p)

In [9]:
lay_BG = pn.Column(
            pn.Spacer(height=11),
            pn.Row(gammaA_slider, gammaB_slider, gammaZ_slider, align="center"), 
            pn.Row(betaA_slider, betaB_slider, betaZ_slider, align="center")
        )
lay_N = pn.Column(
            pn.Row(nXA_slider, nXB_slider), 
            pn.Row(nYA_slider, nYB_slider), 
            pn.Row(nAZ_slider, nBZ_slider)
        )
lay_params = pn.Row(lay_BG, lay_N)

lay_time = pn.Row(t_step_slider, align="center")
lay_init = pn.Column(pn.Row(Xo_button, Xf_button), 
                     pn.Row(Yo_button, Yf_button))
lay_norm = pn.Column(normalize_button, align="center")
pn.Column(lay_params, plotter, lay_time, lay_init, lay_norm)

Column
    [0] Row
        [0] Column
            [0] Spacer(height=11)
            [1] Row(align='center')
                [0] FloatSlider(end=5.0, name='γ_A', start=0.1, value=1.0, width=100)
                [1] FloatSlider(end=5.0, name='γ_B', start=0.1, value=1.0, width=100)
                [2] FloatSlider(end=5.0, name='γ_Z', start=0.1, value=1.0, width=100)
            [2] Row(align='center')
                [0] FloatSlider(end=10.0, name='β_A', start=0.1, value=1.0, width=100)
                [1] FloatSlider(end=10.0, name='β_B', start=0.1, value=1.0, width=100)
                [2] FloatSlider(end=10.0, name='β_Z', start=0.1, value=1.0, width=100)
        [1] Column
            [0] Row
                [0] FloatSlider(end=15.0, name='n_XA', start=0.1, value=5.0, width=100)
                [1] FloatSlider(end=15.0, name='n_XB', start=0.1, value=5.0, width=100)
            [1] Row
                [0] FloatSlider(end=15.0, name='n_YA', start=0.1, value=5.0, width=100)
                [1] FloatSlider(end=15.0, name='n_YB', start=0.1, value=5.0, width=100)
            [2] Row
                [0] FloatSlider(end=15.0, name='n_AZ', start=0.1, value=5.0, width=100)
                [1] FloatSlider(end=15.0, name='n_BZ', start=0.1, value=5.0, width=100)
    [1] ParamFunction(function)
    [2] Row(align='center')
        [0] FloatSlider(end=5.0, name='step time', value=1.8, width=300)
    [3] Column
        [0] Row
            [0] RadioButtonGroup(name='Xo', options=['Xo LOW', 'Xo HIGH'], value='Xo LOW')
            [1] RadioButtonGroup(name='Xf', options=['Xf LOW', 'Xf HIGH'], value='Xf LOW')
        [1] Row
            [0] RadioButtonGroup(name='Yo', options=['Yo LOW', 'Yo HIGH'], value='Yo LOW')
            [1] RadioButtonGroup(name='Yf', options=['Yf LOW', 'Yf HIGH'], value='Yf HIGH')
    [4] Column(align='center')
        [0] RadioButtonGroup(name='normalize', options=['NORMALIZE HALF', ...], value='NORMALIZE HALF')

# Observations: 
- Xo 0, Yo 0 &rarr; Xf 1, Yf 1: As expected, the XOR gate results in zero Z. 
- Xo 1, Yo 1 &rarr; Xf 0, Yf 0: Again, as expected, no Z.
- Xo 0, Yo 0 &rarr; Xf 1, Yf 0: Success! We see that as soon as X is introduced without Y, Z is produced. We also see that there is no B, only A. 
- Xo 0, Yo 0 &rarr; Xf 0, Yf 1: Success! We see that as soon as Y is introduced without X, Z is produced. We also see that there is no A, only B

I am super nervous Panel doesn't work for the reader, so I've included screenshots. I spent a lot of time trying to get the bokeh.models version to work, only to realize that bokeh.models.RaidoButtonGroup doesn't have a "value" attribute, and I feel like changing it to a selector greatly diminishes the quality of the dashboard, and will just hope in the future that Panel fixes its bugs. (Also, I've attached a `XOR_widget.py` file you can try running with `panel serve --show XOR_widget.py`) The results are below. 

<img src="dash1.png" width=600px>
<img src="dash2.png" width=600px>
<img src="dash3.png" width=600px>
<img src="dash4.png" width=600px>

# more combinations!
- Xo 0, Yo 1 &rarr; Xf 1, Yo 0: Perhaps the coolest transition, here we see that low X and high Y stepping to high X and low Y produces Z in an interesting manner, A and B exchange precisely the way we expect them to. At the moment of the step, Z must allow for A to accumulate properly before it reaches its full potential, or else its degradation is overridden.

<img src="dash5.png" width=600px>

<div class="alert alert-block alert-info">
    Whoa this is beautiful. Very nice, thorough analysis!! <br>
    55/55 <br>
    Graded by AR
</div>